<a href="https://colab.research.google.com/github/kimhwijin/KFood_Models/blob/master/KFood_Dataset_Unzip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
import os
import zipfile

In [2]:

DATASET_PATH = Path("/content/drive/MyDrive/Datasets/")
dest_path = DATASET_PATH / "kfood"
if not dest_path.exists():
    with zipfile.ZipFile(DATASET_PATH / "kfood.zip") as kfood:
        zipInfo = kfood.infolist()
        for member in zipInfo:
            try:
                print(member.filename.encode('cp437').decode('euc-kr', 'ignore'))
                member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                kfood.extract(member, dest_path)
            except:
                #print(source_file)
                raise Exception('what?!')


In [ ]:
DATASET_PATH = Path("/content/drive/MyDrive/Datasets/")
root_path = DATASET_PATH / "kfood"

zipfile_paths = list(root_path.glob("*.zip"))

for fname in zipfile_paths:
    fname = fname.stem
    dest_path = root_path
    print(fname)
    with zipfile.ZipFile(root_path / "{}.zip".format(fname), "r") as zf:
        zipInfo = zf.infolist()
        for member in zipInfo:
            member.filename = member.filename.encode('utf8').decode("utf8", "ignore")
            zf.extract(member, dest_path)   


국
기타
김치
나물
떡
만두
면
무침
밥
볶음
쌈
음청류
장
적
장아찌
전
전골
조림
